In [2]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv
import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

In [ ]:
# creating a table from s3

query_text = """--sql
    CREATE TABLE db1.sessions_st_mobile
    (
        `report_date` Date,
        `citizen_id`  Int32,
        `created_at` DateTime,
        `last_use` DateTime,
        `updated_at` DateTime,
        `platform` String,
        `call_enabled` Int16,
        `app` String,
        `logged_in` Int16,
        `timezone` String
    )
    ENGINE = S3('https://storage.yandexcloud.net/dwh-asgard/sessions_st_mobile/year=*/month=*/*.csv','CSVWithNames')
    """

ch.query_run(query_text)

In [16]:
# creating a table for materialized view

query_text = """--sql
    CREATE TABLE db1.sessions_st_mobile_ch
    (
        `report_date` Date,
        `citizen_id`  Int32,
        `created_at` DateTime,
        `last_use` DateTime,
        `updated_at` DateTime,
        `platform` String,
        `call_enabled` Int16,
        `app` String,
        `logged_in` Int16,
        `timezone` String
    )
    ENGINE = MergeTree()
    ORDER BY citizen_id
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [17]:
# creating a materialized view

query_text = """--sql
    CREATE MATERIALIZED VIEW db1.sessions_st_mobile_mv
    REFRESH EVERY 1 DAY OFFSET 3 HOUR RANDOMIZE FOR 1 HOUR TO db1.sessions_st_mobile_ch AS
    SELECT
        *
    FROM db1.sessions_st_mobile
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

____


In [3]:
# creating a materialized view

query_text = """--sql
    SELECT
        *
    FROM db1.sessions_st_mobile_ch
    LIMIT 10
    """

ch.query_run(query_text)

report_date,citizen_id,created_at,last_use,updated_at,platform,call_enabled,app,logged_in,timezone
date,i64,datetime[ns],datetime[ns],datetime[ns],str,i64,str,i64,str
2023-07-26,7,2023-04-03 23:48:50,2023-07-26 18:50:33,2023-07-26 18:50:33,"""android""",1,"""default""",1,"""Europe/Moscow"""
2023-07-31,7,2022-08-23 12:58:48,2023-07-31 22:37:03,2023-07-31 22:37:03,"""ios""",1,"""default""",1,"""Europe/Moscow"""
2023-07-31,7,2023-04-03 23:48:50,2023-07-31 10:24:01,2023-07-31 10:25:17,"""android""",1,"""default""",0,"""Europe/Moscow"""
2023-08-01,7,2022-08-23 12:58:48,2023-08-01 16:30:48,2023-08-01 16:30:48,"""ios""",1,"""default""",1,"""Europe/Moscow"""
2023-08-12,7,2023-04-03 23:48:50,2023-08-12 18:03:19,2023-08-12 18:03:19,"""android""",1,"""default""",1,"""Europe/Moscow"""
2023-08-16,7,2023-04-03 23:48:50,2023-08-16 12:31:47,2023-08-16 12:32:10,"""android""",1,"""default""",0,"""Europe/Moscow"""
2023-08-21,7,2023-04-03 23:48:50,2023-08-21 16:24:26,2023-08-21 16:24:26,"""android""",1,"""default""",1,"""Europe/Moscow"""
2023-08-25,7,2022-08-23 12:58:48,2023-08-25 11:25:20,2023-08-25 11:25:58,"""ios""",1,"""default""",0,"""Europe/Moscow"""
2023-08-25,7,2023-04-03 23:48:50,2023-08-25 11:28:50,2023-08-25 11:28:50,"""android""",1,"""default""",1,"""Europe/Moscow"""


In [12]:
query_text = """--sql
SELECT
	*
FROM db1.`sessions_st_mobile`
WHERE _path LIKE '%/year=2025/month=7/7%' 
   OR _path LIKE '%/year=2025/month=07/07%'
ORDER BY report_date DESC
LIMIT 10
"""

ch.query_run(query_text)

report_date,citizen_id,created_at,last_use,updated_at,platform,call_enabled,app,logged_in,timezone
date,i64,datetime[ns],datetime[ns],datetime[ns],str,i64,str,i64,str
2025-07-07,593190,2022-03-31 22:15:32,2025-07-08 00:15:40,2025-07-08 00:15:40,"""android""",1,"""default""",1,"""Europe/Kaliningrad"""
2025-07-07,593169,2022-03-31 21:54:52,2025-07-07 18:21:22,2025-07-07 18:21:22,"""ios""",0,"""default""",1,"""Europe/Moscow"""
2025-07-07,593161,2022-03-31 21:49:13,2025-07-08 00:04:20,2025-07-08 00:04:20,"""android""",1,"""default""",1,"""Europe/Moscow"""
2025-07-07,593113,2022-03-31 21:10:45,2025-07-07 20:43:54,2025-07-07 20:43:54,"""android""",1,"""default""",1,"""Asia/Krasnoyarsk"""
2025-07-07,593107,2022-03-31 21:03:02,2025-07-07 17:29:11,2025-07-07 17:29:11,"""ios""",1,"""default""",1,"""Asia/Novokuznetsk"""
2025-07-07,593096,2022-03-31 21:02:26,2025-07-07 23:39:51,2025-07-07 23:39:51,"""android""",1,"""default""",1,"""Europe/Moscow"""
2025-07-07,593001,2022-03-31 19:59:18,2025-07-07 22:16:34,2025-07-07 22:16:34,"""android""",1,"""default""",1,"""Europe/Moscow"""
2025-07-07,593092,2022-03-31 20:51:38,2025-07-07 11:17:37,2025-07-07 11:17:37,"""ios""",1,"""default""",1,"""Europe/Samara"""
2025-07-07,593098,2022-03-31 20:55:18,2025-07-07 21:09:36,2025-07-07 21:09:36,"""ios""",1,"""default""",1,"""Europe/Moscow"""


___
## Drop the Table

In [14]:
query_text = """
    DROP TABLE db1.sessions_st_mobile_ch
    """
ch.query_run(query_text)

elapsed_ns,query_id
i64,str
1781355,"""438cbbba-80de-481f-8b28-b569fd…"


In [15]:
query_text = """
    DROP TABLE db1.sessions_st_mobile_mv
    """
ch.query_run(query_text)

elapsed_ns,query_id
i64,str
8218745,"""38234a96-35b7-4751-a70a-1d985e…"


___

## Refreshing the data

In [ ]:

query_text = """
SYSTEM REFRESH VIEW db1.sessions_st_mobile_mv
"""

ch.query_run(query_text)